In [1]:
import torch
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path
from torchvision import transforms
import numpy as np

# Costruzione di un oggetto dataset personalizzato

Consideriamo il dataset a [questo link] (http://people.csail.mit.edu/torralba/code/spatialenvelope/spatial_envelope_256x256_static_8outdoorcategories.zip)

2688 immagini a colori 256x256 suddivise in 8 classi a secondo del tipo di scena del ritratto
- forest
- highway
- insidecity
- mountain
- opencountry
- street
- tallbuilding

All'interno della cartella sono presenti 3 file testo
- `train.txt`: 2188 immagini di training con relative etichette in formato numerico (0-7)
- `test.txt`: contiene i nomi delle rimanenti 500 immagini di testing con le relative etichette in formato numerico
- `classes.txt`: nomi delle 3 classi. La i-esima conterrà il nome della classe i-esima

Costruiamo di conseguenza un oggetto Dataset che ci permetta di caricare le immagini di training e test. Ciò si può fare in maniera naturale in PyTorch ereditando dalla classe Dataset. Ogni oggetto deve contenere almeno
- costruttore
- metodo __len__ che restituisce il numero di elementi contenuti nel dataset
- metodo __getitem__ che prende in input un indice i e restituisce l'i-esimo elemento del dataset


In [2]:
class ScenesDataset(Dataset):
    """Implementa l'oggetto scenesDataset che ci permette di caricare le immagini del dataset 8 scenes"""
    def __init__(self, base_path, txt_list, transform=None):
        """Input:
        base_path: path della cartella contenente le immagini
        txt_list: il path al file del testo contenente la lista delle immagini con le relative etichette. Ad esempio train.txt o test.txt
        transform: implementeremo il dataset in modo che esso supporti le trasformazioni """
        # conserviamo il path alla cartella contenente le immagini
        self.base_path = base_path
        # carichiamo la lista dei file
        # sarà una matrice con n righe (n di immagini) e 2 colonne (path, etichetta)

        self.images = np.loadtxt(txt_list, dtype=str, delimiter=',')
        # conserviamo il riferimento alla trasformazione da applicare
        self.transform = transform

    def __getitem__(self, index):
        # recuperiamo il path dell'immagine di indice index e la relativa etichetta
        f,c = self.images[index]

        # carichiamo l'immagine utilizzando PIL
        im = Image.open(path.join(self.base_path, f))

        # se la trasformazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)
        
        # converto l'etichetta in intero
        label = int(c)

        #restituisco un dizionario contenente immagine etichetta
        return {'image': im, 'label': label}

    # restituisce numero di campioni: la linghezza della lista images
    def __len__(self):
        return len(self.images)

Istanzio il dataset per caricare dei dati

In [3]:
dataset = ScenesDataset('8scenes','8scenes/train.txt',transform=transforms.ToTensor())
sample = dataset[0]
#l'immagine è 3 x 256 x 256 perché è una immagine a colori
print(sample['image'].shape)
print(sample['label'])

torch.Size([3, 256, 256])
1


Le immagini sono di dimensioni 256x256. Per ridurre i tempi computazionali, potremmo voler lavorare con immagini più piccole usando trasformazione Resize

In [4]:
transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor()])
dataset = ScenesDataset('8scenes','8scenes/train.txt', transform=transform)
sample = dataset[0]
print(sample['image'].shape, sample['label'])

torch.Size([3, 32, 32]) 1


Per poter normalizzare i dati calcoliamo medie e varianza di tutti i pixel contenuti nelle immagini del dataset. Nel caso di immagini a colori vanno calcolati canale per canale:

In [5]:
dataset = ScenesDataset('8scenes', '8scenes/train.txt', transform=transforms.ToTensor())
m = np.zeros(3)
for sample in dataset:
    m+= sample['image'].sum(1).sum(1).numpy() ## accumulo somma dei pixel canale per canale

# divido per il numero di immagini moltiplicato per il n di pixel
m = m/(len(dataset)*256*256)

# procedura simile per calcolare dev std
s = np.zeros(3)
for sample in dataset:
    s+=((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1).numpy()

s=np.sqrt(s/(len(dataset)*256*256))


print("Medie:", m, "\n dev std:", s)

Medie: [0.42478886 0.45170452 0.4486708 ] 
 dev std: [0.25579566 0.24652381 0.27658252]


Inseriamo la corretta normalizzazione tra le trasformazioni

In [6]:
transform = transforms.Compose([transforms.Resize(32), transforms.ToTensor(), transforms.Normalize(m,s), transforms.Lambda(lambda x: x.view(-1))])

dataset = ScenesDataset('8scenes', '8scenes/train.txt', transform=transform)

print(dataset[0]['image'].shape, dataset[0]['label'])

torch.Size([3072]) 1
